In [1]:
import os
import fitz
from tqdm import tqdm
from shutil import copy
import zipfile
from shutil import move
from ExtractPdf import *
from Sinopac import Sinopac
from Ibf import Ibf
from Ctbc import Ctbc
from Fubon import Fubon
from Yuanta import Yuanta

In [2]:
def handler_test(directory_path, date, processed_path):
    count_false = 0
    count_true = 0
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            full_path = os.path.join(directory_path, filename)
            old_name = os.path.splitext(filename)[0] # get the original filename without extension      
            # Split the base name into its components
            components = old_name.split('_')
            # Extract the components
            stock_num = components[0]
            stock_name = components[1]
            advisor_name = components[2]
            old_rate = components[3]
            summary = 'NULL'
            pdf_handler = ExtractPdf()
            if '永豐' in advisor_name:
                new_rate = pdf_handler.sinopac(full_path)
            elif '國票' in advisor_name:
                new_rate = pdf_handler.ibf(full_path)
            elif 'CTBC' in advisor_name or '中信' in advisor_name:
                new_rate = pdf_handler.ctbc(full_path)
            elif '台新' in advisor_name:
                new_rate = pdf_handler.taishin(full_path)
            elif '宏遠' in advisor_name:
                new_rate = pdf_handler.honsec(full_path)
            elif '元大' in advisor_name:
                new_rate = pdf_handler.yuanta(full_path)
            elif '元富' in advisor_name:
                new_rate = pdf_handler.masterlink(full_path)
            elif '富邦' in advisor_name:
                new_rate = pdf_handler.fubon(full_path)
            elif '統一' in advisor_name:
                new_rate = pdf_handler.pscnet(full_path)
            elif '群益' in advisor_name or 'capital' in advisor_name:
                new_rate = pdf_handler.capital(full_path)
            elif '第一金' in advisor_name or '一銀' in advisor_name:
                new_rate = pdf_handler.ffhc(full_path)  
            elif '日昇' in advisor_name:
                new_rate = pdf_handler.jihsun(full_path) 
            elif '玉山' in advisor_name:
                new_rate = pdf_handler.esun(full_path) 
            elif '國泰' in advisor_name:
                new_rate = pdf_handler.cathay(full_path) 
            elif '兆豐' in advisor_name:
                new_rate = pdf_handler.mega(full_path) 
            elif '福邦' in advisor_name:
                new_rate = pdf_handler.gfortune(full_path)      
            elif '康和' in advisor_name:
                new_rate = pdf_handler.concords(full_path) 
            elif 'MS' in advisor_name or 'ms' in advisor_name:
                new_rate = pdf_handler.morganstanley(full_path)
            elif 'CLSA' in advisor_name or 'cl' in advisor_name:
                new_rate = pdf_handler.clsa(full_path)  
            elif 'citi' in advisor_name:
                new_rate = pdf_handler.citi(full_path) 
            elif 'GS' in advisor_name or '高盛' in advisor_name or 'gs' in advisor_name:
                new_rate = pdf_handler.goldmansachs(full_path)  
            elif 'hsbc' in advisor_name or 'HSBC' in advisor_name:
                new_rate = pdf_handler.hsbc(full_path)  
            elif 'jpm' in advisor_name or '摩根大通' in advisor_name:
                new_rate = pdf_handler.jpmorgan(full_path)  
            elif 'NMR' in advisor_name or '野村' in advisor_name:
                new_rate = pdf_handler.nomura(full_path)   
            else :
                # new_rate = 'NULL'
                new_rate = 'wrong'
            new_name = f'{stock_num}_{stock_name}_{date}_{advisor_name}_{new_rate}_{summary}.pdf'
            new_path = os.path.join(processed_path, new_name)
            # copy(full_path, new_path) # copy the processed file to the output directory
            if new_rate == 'NULL':
                # count_true += 1
                # print(f'filename = {filename}')
                # print(f'advisor_name={advisor_name}, new_rate={new_rate}')
                pass
            elif new_rate == 'wrong':
                count_true += 1
                print(f'filename = {filename}')
                print(f'advisor_name={advisor_name}, new_rate={new_rate}')
                # pass
            elif new_rate == old_rate:
                count_true += 1
            elif new_rate in old_rate:
                count_true += 1
            elif old_rate in new_rate:
                count_true += 1
            else:
                count_true += 1
                # print(f'filename = {filename}')
                # print(f'advisor_name={advisor_name}, new_rate={new_rate}')
    count_false = len(os.listdir(directory_path)) - count_true
    success_rate = round((count_true/len(os.listdir(directory_path))), 3)*100
    print(f'共{len(os.listdir(directory_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')


In [3]:
def handler_find_advisor(directory_path):
    count_false = 0
    count_true = 0
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            full_path = os.path.join(directory_path, filename)
            old_name = os.path.splitext(filename)[0] # get the original filename without extension      
            # Split the base name into its components
            # Extract the components
            advisor = 'NULL'
            pdf_handler = ExtractPdf()
            ADVISOR_COMPANIES = [pdf_handler.sinopac, pdf_handler.ibf, pdf_handler.ctbc, 
                                 pdf_handler.taishin, pdf_handler.honsec, pdf_handler.yuanta, 
                                 pdf_handler.masterlink, pdf_handler.fubon, pdf_handler.pscnet, 
                                 pdf_handler.capital, pdf_handler.ffhc, pdf_handler.jihsun, 
                                 pdf_handler.esun, pdf_handler.cathay, pdf_handler.mega, 
                                 pdf_handler.gfortune, pdf_handler.concords, pdf_handler.morganstanley,
                                 pdf_handler.clsa, pdf_handler.citi, pdf_handler.goldmansachs, 
                                 pdf_handler.hsbc, pdf_handler.jpmorgan, pdf_handler.nomura]
            for company in ADVISOR_COMPANIES:
                new_rate = company(full_path)
                if pdf_handler.advisor != 'NULL':
                    advisor = pdf_handler.advisor
                    break
            if advisor == 'NULL':
                # print(full_path)
                count_false+=1
    if len(os.listdir(directory_path))>0:
        count_true = len(os.listdir(directory_path)) - count_false
        success_rate = round((count_true/(len(os.listdir(directory_path)))), 3)*100
        print(f'{directory_path}, 共{len(os.listdir(directory_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')


In [5]:
root = '../history_data/file/元大/'
count_false_rating = 0
count_true_rating = 0
count_false_tp = 0
count_true_tp = 0
count_false_author = 0
count_true_arthor = 0
count_false_summary = 0
count_true_summary = 0

for filename in os.listdir(root):
    file_path = os.path.join(root, filename)
    pdfReader = Yuanta(file_path)
    rating = pdfReader.get_rating_process()
    target_price = pdfReader.get_target_price_process()
    author = pdfReader.get_author_process()
    summary = pdfReader.get_summary_process()

    if rating == 'NULL':
        count_false_rating+=1
        
    if target_price == 'NULL':
        count_false_tp+=1

    if author == 'NULL':
        print(filename)
        print(rating)
        print(target_price)
        print('author_1', pdfReader.author_1)
        print('author_2', pdfReader.author_2)
        print(summary)
        count_false_author+=1

    if summary == 'NULL':
        count_false_summary+=1

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_rating
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_rating}件, 成功率:{success_rate}%')

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_tp
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_tp}件, 成功率:{success_rate}%')

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_author
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_author}件, 成功率:{success_rate}%')

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_summary
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_summary}件, 成功率:{success_rate}%')


1101_台泥_2019-03-25_元大_Taiwan_Cement_1101_TT_-_Yuanta_CVN190324-C-YCMC.pdf
NULL
38.70
author_1 分析師聲明及重要披露事項請參見附錄 A。
author_2 
2019 年 3 月 24 日 
 
台灣水泥 (1101 TT) 
2018 年第四季財報低於預期，但股利政策隱含獲
利成長空間 
 
事件
1101_台泥_2019-06-19_元大_Taiwan_Cement_1101_TT_-_Yuanta_EXN190619-C-YCMC.pdf
NULL
45.30
author_1 分析師聲明及重要披露事項請參見附錄 A。
author_2 
2019 年 6 月 19 日 
 
台灣水泥 (1101 TT) 
元大亞洲投資創富論壇重點摘要 
請點此參與今年 AsiaMoney 票選，將您寶貴的一票投給元大！ 
事件 
台泥於 6 月 19 日參加元大亞洲投資創富論壇，與投資人分享獲利
1216_統一_2018-03-29_元大_Uni-President_1216_TT_-_Yuanta_EXN180329-C-YCMC.pdf
NULL
68.00
author_1 分析師聲明及重要披露事項請參見附錄 A。
author_2 
2018 年 3 月 29 日 
 
統一企業 (1216 TT) 
股利帶來正面驚喜 
事件 
統一 2017 年每股盈餘 7.01 元，優於本中心預估之 3.09 元，係
受惠於上海星巴克交易案的一次性處分利益。集團亦宣佈配發現金
1305_華夏_2018-03-13_元大_China_General_Plastics_1305_TT_-_Yuanta_EX180313-C-YCMC.pdf
NULL
32.75
author_1 分析師聲明及重要披露事項請參見附錄 A。
author_2 
2018 年 3 月 13 日 
 
華夏 (1305 TT) 
費用與匯損拖累 2017 年第四季財報低於預期；
2018 年第一季獲利估值可望上修 
事件
1309_台達化_2019-05-28_元大_Taita_Chemical_1309_TT_-_Yuanta_IN190527-C-YCMC.pdf
NULL
NULL
author_1 分析師聲

In [8]:
root = '../history_data/file/富邦/'
count_false_rating = 0
count_true_rating = 0
count_false_tp = 0
count_true_tp = 0
count_false_author = 0
count_true_arthor = 0
count_false_summary = 0
count_true_summary = 0
for filename in os.listdir(root):
    file_path = os.path.join(root, filename)
    pdfReader = Fubon(file_path)
    rating = pdfReader.get_rating_process()
    target_price = pdfReader.get_target_price_process()
    author = pdfReader.get_author_process()
    summary = pdfReader.get_summary_process()

    if rating == 'NULL':
        # print(filename)
        # print(rating)
        # print(target_price)
        # print(author)
        # print(summary)
        count_false_rating+=1
        
    if target_price == 'NULL':
        count_false_tp+=1

    if author == 'NULL':
        count_false_author+=1

    if summary == 'NULL':
        count_false_summary+=1
        
if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_rating
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_rating}件, 成功率:{success_rate}%')

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_tp
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_tp}件, 成功率:{success_rate}%')

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_author
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_author}件, 成功率:{success_rate}%')

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_summary
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_summary}件, 成功率:{success_rate}%')


../history_data/file/富邦/, 共717件, 成功685件, 失敗32件, 成功率:95.5%
../history_data/file/富邦/, 共717件, 成功479件, 失敗238件, 成功率:66.8%
../history_data/file/富邦/, 共717件, 成功73件, 失敗644件, 成功率:10.2%
../history_data/file/富邦/, 共717件, 成功639件, 失敗78件, 成功率:89.1%


In [5]:
root = '../history_data/file/國票/'
count_false = 0
count_true = 0
for filename in os.listdir(root):
    file_path = os.path.join(root, filename)

    pdfReader = Ibf(file_path)
    rating = pdfReader.get_rating_process()
    target_price = pdfReader.get_target_price_process()
    author = pdfReader.get_author_process()
    summary = pdfReader.get_summary_process()

    if summary == 'NULL':
        print(filename)
        print(rating)
        print(target_price)
        print(author)
        print(summary)
        count_false+=1
        
        
if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')


1519-華城-2023_03_08-國票-買進，目標價NT$7.pdf
買進
78
周品三
NULL
1795_美時_國票_買進.pdf
買進
305
周品三
NULL
2020_美亞_2019-04-26_國票_國票04262019總經評論_日本央行修改前瞻指引，指出將維持超低利率至2020春，並下調GDP預估.pdf
NULL
前瞻指引，維持超低利

NULL
2231_為升_2018-03-07_國票_2231為升03072018.pdf
買進
401
陳宜均
NULL
2301_光寶科_2018-10-09_國票_2301光寶科10092018.pdf
區間操作
36~40
周宛瑩
NULL
2349_錸德_2018-08-15_國票_2349錸德08152018.pdf
區間操作
12.4
黃琳証
NULL
2360_致茂_2019-02-25_國票_國票2360致茂02252019.pdf
區間操作
112~140
陳蘭蘭
NULL
2360_致茂_2019-05-02_國票_國票2360致茂05022019.pdf
區間操作
128~153
陳蘭蘭
NULL
2367_燿華_2018-11-21_國票_2367燿華11212018.pdf
區間操作
10~14
黃琳証
NULL
2367_燿華_2019-04-25_國票_國票2367燿華04252019.pdf
區間操作
17.5~22
黃琳証
NULL
2383_台光電_2018-12-20_國票_2383台光電12202018.pdf
區間操作
52~72
黃琳証
NULL
2402_毅嘉_2018-07-13_國票_2402毅嘉07132018.pdf
區間操作
19.5
黃琳証
NULL
2439_美律_2018-07-31_國票_2439美律07312018.pdf
區間操作
175
黃琳証
NULL
2439_美律_2019-06-12_國票_國票2439美律06122019.pdf
買進
198
周宛瑩
NULL
2481_強茂_2019-04-22_國票_國票2481強茂04222019.pdf
區間操作
22~27
陳蘭蘭
NULL
3008_大立光_2018-09-19_國票_3008大立光09192018.pdf
買進
5300
周宛瑩
NULL
3016_嘉晶_2018-10

In [5]:
# specify the path to the compressed file
zip_path = 'file'

# specify the path to the directory where the extracted files will be saved
extract_root = r'output\extract'
processed_root = r'output\processed'
if not os.path.exists(extract_root):
    os.makedirs(extract_root)
if not os.path.exists(processed_root):
    os.makedirs(processed_root) 
# create a ZipFile object and extract the contents to the output directory
for directory in os.listdir(zip_path):
    directory_path = os.path.join(zip_path, directory)
    extract_path = os.path.join(extract_root, os.path.splitext(directory)[0])
    with zipfile.ZipFile(directory_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)


FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: 'file'

In [ ]:
all_pdf_root = r'../history_data/file'

In [ ]:
for date_path in os.listdir(all_pdf_root):
    directory_date = os.path.join(all_pdf_root, date_path)
    handler_find_advisor(directory_date)
        

TypeError: ExtractPdf.__init__() missing 1 required positional argument: 'file_path'

In [ ]:
for date_path in os.listdir(extract_root):
    directory_date = os.path.join(extract_root, date_path)
    processed_date = os.path.join(processed_root, date_path)
    date = date_path
    # print(date)
    for directory in os.listdir(directory_date):
        directory_path = os.path.join(directory_date, directory)
        # print(directory_path)zz
        processed_path = os.path.join(processed_date, directory)
        if not os.path.exists(processed_path):
            os.makedirs(processed_path) 
        if directory == 'test':
            handler_find_advisor(directory_path)
        # if directory == '2':
        #     handler_2(directory_path, date, processed_path)
        